In [1]:
import matplotlib.pyplot as plt
from src.utils.plotting import init_plot_style
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
def coef(c,A):
    """Compute the coefficents for the given function, evaluate at x =c, with input parameter A"""
    
    fc0 = 1/(1+np.exp(-c))
    fc1 = np.exp(-c)/(1+np.exp(-c))**2
    fc2 = -np.exp(-c)/(1+np.exp(-c))**2 + 2*np.exp(-2*c)/(1+np.exp(-c))**3
    fc3 = np.exp(-c)/(1+np.exp(-c))**2 - 6*np.exp(-2*c)/(1+np.exp(-c))**3 + 6*np.exp(-3*c)/(1+np.exp(-c))**4
        
    a0 = fc0 - fc1*c + 0.5*fc2*(c**2)-(1/6)*fc3*(c**3) + 0.25*(fc2-c*fc3)*(A**2)
    a1 = (fc1 -fc2*c+0.5*fc3*(c**2))*A + 0.75*(1/6)*fc3*(A**3)
    a2 = 0.25*(fc2-fc3*c)*(A**2)
    a3 = 0.25*(1/6)*fc3*(A**3)
        
    return [a0, a1, a2, a3]

def system(x, A):
    """Original system f(x[n])"""
    theta = np.pi*2/5
    y = 1/(1+np.exp(-A*cos(theta*x)))
    return y

def approx_sys(x, a):
    """Evaluate Taylor approximation"""
    theta = np.pi*2/5
    y_hat = a[0] + a[1]*cos(theta*x)+a[2]*cos(2*theta*x) + a[3]*cos(3*theta*x)
    return y_hat



In [3]:
#alpha = np.zeros([1,4])
alpha_0_1 = coef(0,1)
alpha_ln_1 = coef(np.log(2),1)

alpha_0_3 = coef(0,1.5)
alpha_ln_3 = coef(np.log(2),1.5)

#Evaluate the system
x = np.arange(-3,3,0.05)
y_1 = system(x, 1)
y_3 = system(x, 3)

y_0_1 = approx_sys(x,alpha_0_1)
y_ln_1 = approx_sys(x,alpha_ln_1)
y_0_3 = approx_sys(x,alpha_0_3)
y_ln_3 = approx_sys(x,alpha_ln_3)


plt.figure(figsize=(8,5))
plt.plot(x, y_1, label='Original system')
plt.plot(x, y_0_1, label='Taylor approxiamtaion at c = 0')
plt.plot(x, y_ln_1, label='Taylor approxiamtaion at c = ln(2)')
plt.legend()
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('A = 1')
plt.grid()
plt.tight_layout()
plt.show()

plt.figure(figsize=(8,5))
plt.plot(x, y_3, label='Original system')
plt.plot(x, y_0_3, label='Taylor approxiamtaion at c = 0')
plt.plot(x, y_ln_3, label='Taylor approxiamtaion at c = ln(2)')
plt.legend()
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('A = 3')
plt.grid()
plt.tight_layout()
plt.show()
